In [1]:
import numpy as np
import pandas as pd
import string
import re

#sklearn imports
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.grid_search import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

#scipy imports
from scipy.sparse import hstack

#Visualization imports
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec 
import bokeh
#! pip install bokeh

# target classes
target_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

/usr/local/lib/python3.4/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
# read frames localy through csv
train_df = pd.read_csv("../data/train.csv")
test_df = pd.read_csv("../data/test.csv")

# # Random index generator for splitting training data
# # Note: Each rerun of cell will create new splits.
# randIndexCut = np.random.rand(len(train_df)) < 0.7

# #S plit up data
# test_data = test_df["comment_text"]
# dev_data, dev_labels = train_df[~randIndexCut]["comment_text"], train_df[~randIndexCut][target_names]
# train_data, train_labels = train_df[randIndexCut]["comment_text"], train_df[randIndexCut][target_names]
# tiny_data,small_data = train_df[:200]["comment_text"],train_df[:1000]["comment_text"]
# tiny_labels,small_labels = train_df[:200][target_names],train_df[:1000][target_names]

train_data = train_df[:]["comment_text"]

# print('total training observations:', train_df.shape[0])
# print('training data shape:', train_data.shape)
# print('training label shape:', train_labels.shape)
# print('dev label shape:', dev_labels.shape)
# print ('labels names:', target_names)

In [3]:
print(train_df.head())
print(test_df.head())

                 id                                       comment_text  toxic  \
0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
1  000103f0d9cfb60f  D'aww! He matches this background colour I'm s...      0   
2  000113f07ec002fd  Hey man, I'm really not trying to edit war. It...      0   
3  0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...      0   
4  0001d958c54c6e35  You, sir, are my hero. Any chance you remember...      0   

   severe_toxic  obscene  threat  insult  identity_hate  
0             0        0       0       0              0  
1             0        0       0       0              0  
2             0        0       0       0              0  
3             0        0       0       0              0  
4             0        0       0       0              0  
                 id                                       comment_text
0  00001cee341fdb12  Yo bitch Ja Rule is more succesful then you'll...
1  0000247867823ef7  ==

In [4]:
# from http://norvig.com/spell-correct.html
import re
from collections import Counter

def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('../data/big.txt').read()))

def norvig_P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def norvig_correction(word): 
    "Most probable spelling correction for word."
    return max(norvig_candidates(word), key=norvig_P)

def norvig_candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [ ]:
# Functions to support finding and correcting spellings
# using pyenchant for spell checking
from enchant import DictWithPWL
from enchant.checker import SpellChecker
import difflib
# import splitter # not useful, does a worse job than my implementation

# mywords.txt currently contains:
# - list of firstnames and surnames gathered from internet searches
# http://www.birkenhoerdt.net/surnames-all.php?tree=1
# www.tysto.com/uk-us-spelling-list.html
my_dict=DictWithPWL('en_US', "../data/mywords.txt")
my_checker = SpellChecker(my_dict)

punctuation = "[\!\?\"\#\$\%\&\(\)\*\+\,\.\/\:\;\<\=\>\?\@\[\]\^\_\`\{\|\}\~\']"

# list of swear words correctly spelt courtesy of https://www.noswearing.com/

def my_preprocessor(textblock):
    # u -> you
    # c -> see
    # k -> okay
    return_words = textblock

#     return_words = re.sub(r"[^A-Za-z0-9,!?*.;’´'\/]", " ", return_words)
    return_words = re.sub(r"[^A-Za-z0-9]", " ", return_words)
    return_words = re.sub(r","," ",return_words)
    return_words = re.sub(r"\.\.+"," ",return_words)
    return_words = re.sub(r"\."," ",return_words)
    return_words = re.sub(r"\("," ", return_words)
    return_words = re.sub(r"\)"," ", return_words)
    return_works = re.sub(r"\;", " ", return_words)
    return_words = re.sub(r":"," ", return_words)
    return_words = re.sub(r"´", "'", return_words)
    return_words = re.sub(r"`", "'", return_words)
    return_words = re.sub(r"''+", "'", return_words)
    return_words = re.sub(r" '", " ", return_words)
    return_words = re.sub(r"' ", " ", return_words)
    return_words = re.sub(r"\"", " ", return_words)
    return_words = re.sub(r"\/", " ", return_words)
    return_words = re.sub(r"\!\!+", "!!", return_words)
    return_words = re.sub(r"\?\?+", "?!", return_words)
    return_words = re.sub(r"\!", " !", return_words)
    return_words = re.sub(r"\?", " ?", return_words)
    return_words = re.sub(r"\b\d+\b", "999", return_words)
    # slang and abbreviations, need to be aware of capitolization and spaces
    return_words = re.sub(r"[Ww]on't", "will not", return_words)
    return_words = re.sub(r"n't", " not", return_words)
    return_words = re.sub(r"'s\b", " is", return_words)
    return_words = re.sub(r"\b[Aa]bt\b", "about", return_words)
    return return_words

def trysplit(word, verbose=False):
    split_candidates = []
    max_proba = 0.0
    for i in range(1,len(word)):
        # I will only allow single letters of 'a' and 'i', all others ignored.  Pyenchant allows for
        # any single letter to be a legitimate word, and so too does norvig.  The dictionary defines
        # them as nouns that represent the letter, however even though several can be used in slang
        # (e.g. k->okay, c->see, u->you) using them in conjoined words would make the splitting far
        # too difficult and also human understanding much more difficult #howucthisk, u c?
        if (len(word[:i]) != 1 or (word[:i].lower() == 'a' or word[:i].lower() == 'i')) and (
            len(word[i:]) != 1 or (word[i:].lower() == 'a' or word[i:].lower() == 'i')):
            if my_checker.check(word[:i]) and my_checker.check(word[i:]):
                norvig_score = norvig_P(word[:i]) + norvig_P(word[i:])
                if norvig_score > max_proba:
                    max_proba = norvig_score
                    split_candidates = [word[:i],word[i:]]
    for i in range(1,len(word)):
        for j in range(i+1,len(word)):        
            if (len(word[:i]) != 1 or (word[:i].lower() == 'a' or word[:i].lower() == 'i')) and (
                len(word[i:j]) != 1 or (word[i:j].lower() == 'a' or word[i:j].lower() == 'i')) and (
                len(word[i:]) != 1 or (word[i:].lower() == 'a' or word[i:].lower() == 'i')):
                
                if my_checker.check(word[:i]) and my_checker.check(word[i:j]) and my_checker.check(word[j:]):
                    norvig_score = norvig_P(word[:i]) + norvig_P(word[i:j]) + norvig_P(word[j:])
                    if norvig_score > max_proba:
                        max_proba = norvig_score
                        split_candidates = [word[:i],word[i:j],word[j:]]
    for i in range(1,len(word)):
        for j in range(i+1,len(word)):
            for k in range(j+1,len(word)):
                if (len(word[:i]) != 1 or (word[:i].lower() == 'a' or word[:i].lower() == 'i')) and (
                    len(word[i:j]) != 1 or (word[i:j].lower() == 'a' or word[i:j].lower() == 'i')) and (
                    len(word[j:k]) != 1 or (word[j:k].lower() == 'a' or word[j:k].lower() == 'i')) and (
                    len(word[k:]) != 1 or (word[k:].lower() == 'a' or word[k:].lower() == 'i')):
                    verbose and print("making it here with i=%s j=%s k=%s %s  max_proba=%d" %(word[:i],word[i:j],word[j:k],word[k:], max_proba))
                    verbose and print("lengths are %d %d %d %d" % (len(word[:i]), len(word[i:j]),len(word[j:k]),len(word[k:])))
                    if my_checker.check(word[:i]) and my_checker.check(word[i:j]) and my_checker.check(word[j:k]) and my_checker.check(word[k:]):
                        verbose and print('found words ' + word[i:] + ' ' + word[k:])
                        norvig_score = norvig_P(word[:i]) + norvig_P(word[i:j]) + norvig_P(word[j:k]) + norvig_P(word[k:])
                        if norvig_score > max_proba:
                            verbose and print("found higher probability %d with %s %s %s %s" % (norvig_score, word[:i], word[i:j], word[j:k], word[k:]))
                            max_proba = norvig_score
                            split_candidates = [word[:i],word[i:j],word[j:k],word[k:]]
    return split_candidates

def get_best_candidates(word):
    best_words = []
    best_ratio = 0
    a = set(my_checker.suggest(word))
    for b in a:
        if not '-' in b:
            tmp = difflib.SequenceMatcher(None, word, b).ratio()
            if tmp > best_ratio:
                best_words=[b]
                best_ratio=tmp
            elif tmp == best_ratio:
                best_words.append(b)
    return best_words
    
def fix_spellings(textblock, verbose=False):
    textblock = re.sub(r'([a-zA-Z_])\1+', r'\1\1',textblock)
    words = textblock.split()
    return_list = []

    for word in words:
        if my_checker.check(word) or my_checker.check(word.lower()) or word in punctuation or\
            any(i.isdigit() or i == '_' for i in word) or (word[-1].lower() == 's' and my_checker.check(word[:-1].lower())):
            return_list.append(word)
        elif len(word) < 100:            
            candidates = get_best_candidates(word)
            if len(candidates) == 1:
                return_list.append(candidates.pop())
            elif len(candidates) > 1:
                # try another spell checker
                nv_candidates = norvig_candidates(word)
                tmp_set = set(nv_candidates).intersection(set(candidates))
                if len(tmp_set) == 1:
                    # only 1 overlap, should be correct
                    return_list.append(tmp_set.pop())
                elif len(nv_candidates) == 1 and next(iter(nv_candidates)) == word:
                        # this is suspicious, pyenchants' "suggest" method always returns something, however if
                        # norvigs method cannot find a suitable match within a short distance then it simply
                        # returns the orignal word.  This section is for potentially conjoined words
                        tmp_list=trysplit(word)

                        # If we get back a list of split words then use these
                        if len(tmp_list) != 0:
                            return_list.extend(tmp_list)
                        else:
                            return_list.append(word)
                else:
                    # arbitrary now, just going to use the first one found from pyenchant, even though
                    # I have seen norvig get the correct word sometimes when pyenchant gets it wrong
                    return_list.append(candidates[0])
            else:
                nv_candidates = norvig_candidates(word)
                if len(nv_candidates) > 0:
                    return_list.append(nv_candidates[0])
                else:
                    return_list.append(word)
        else
            return_list.append(word)
    return " ".join(return_list)

# myword='In a long discussion about thisismessedup what should I do askd'
# print(fix_spellings(myword))

In [6]:
count_vect_stop_words = CountVectorizer()
X_train_counts_stop_words = count_vect_stop_words.fit_transform(train_data)
print(len(count_vect_stop_words.vocabulary_))

189775


In [ ]:
index=1
for word in count_vect_stop_words.get_feature_names():
    try:
        return_word = fix_spellings(word)[0]
        if word != return_word:
            print("%s : %s " % (word, return_word))
#         if index % 1000 == 0:
#             print("Number of words processed %d" % (index))
#         index += 1
    except:
        print('Word %s caused exception, returned %s' % (word, fix_spellings(word)))
        
print('done')

00 : 0 
000 : 0 
0000 : 0 
00000 : 0 
000000 : 0 
0000000 : 0 
00000000 : 0 
0000000027 : 0 
00000001 : 0 
00000003 : 0 
00000050 : 0 
000001 : 0 
000002 : 0 
000002000004000008000016 : 0 
0000030422 : 0 
0000035 : 0 
000023405011 : 0 
00004 : 0 
000045 : 0 
00007632 : 0 
000080 : 0 
00008b : 0 
00009c : 0 
0000z : 0 
0001 : 0 
000111002 : 0 
00012 : 0 
000131088 : 0 
0002tree : 0 
0003 : 0 
000319821 : 0 
00033 : 0 
00035 : 0 
000352 : 0 
00037 : 0 
00038 : 0 
0003º : 0 
0004 : 0 
000500 : 0 
00052 : 0 
00053 : 0 
0006 : 0 
00075 : 0 
00098 : 0 
000_bucks : 0 
000b01c5b9f8 : 0 
000feet : 0 
000ft : 0 
000http : 0 
000hz : 0 
000k : 0 
000kg : 0 
000km : 0 
000lb : 0 
000poland2 : 0 
000s : 0 
000shp : 0 
000soviet : 0 
000th : 0 
000unknownunknown : 0 
000unknownunknownhungary : 0 
000unknownunknownitaly : 0 
000unknownunknownromania3 : 0 
000unknownunknowntotal : 0 
000v : 0 
000x : 0 
000z : 0 
001 : 0 
0010 : 0 
00107 : 0 
0010steve : 0 
0011 : 0 
001200405140111 : 0 
00122 : 0 
00

0chmq6aewcq : 0 
0d5c4ebc69fbb9bd9f37b47f8b0e1cef : 0 
0e394f018fc9ff00vgnvcm1000002c567798rcrd : 0 
0em : 0 
0f : 0 
0f78 : 0 
0fb3382ee6ad1e46 : 0 
0h : 0 
0j0i10l9 : 0 
0j1j0j3j3 : 0 
0k : 0 
0l4 : 0 
0m : 0 
0m6hr : 0 
0mf : 0 
0mg : 0 
0n : 0 
0pb : 0 
0pointer : 0 
0px : 0 
0q5pxmir2u : 0 
0qs : 0 
0r : 0 
0rganizashun : 0 
0rlp0a2qso8 : 0 
0rr : 0 
0rz : 0 
0st : 0 
0t : 0 
0th : 0 
0therf : 0 
0v : 0 
0waldo : 0 
0wn : 0 
0x : 0 
0x00 : 0 
0x01 : 0 
0x0104 : 0 
0x10 : 0 
0x66 : 0 
0x6a : 0 
0x80 : 0 
0xc0 : 0 
0yg5en4k344 : 0 
0ºe : 0 
0ºn : 0 
0ºs : 0 
0ºw : 0 
10 : 1 
100 : 1 
1000 : 1 
10000 : 1 
100000 : 1 
1000000 : 1 
10000000 : 1 
100000000 : 1 
1000000000 : 1 
10000000000000 : 1 
100000000000000000 : 1 
1000000000000000000 : 1 
100000000000000000000000000000 : 1 
10000000000000000000000000000000000000000000 : 1 
1000000000x : 1 
10000015 : 1 
100000179702430 : 1 
100000x : 1 
100002283868363 : 1 
100005188847302 : 1 
100012570199 : 1 
100012987475 : 1 
100013198462 : 1 

13kb : 1 
13kx : 1 
13l : 1 
13music : 1 
13nd : 1 
13newsb : 1 
13qwi : 1 
13t08 : 1 
13th : 1 
13w : 1 
13we : 1 
13yr : 1 
14 : 1 
140 : 1 
1400 : 1 
14000 : 1 
14000bc : 1 
14000store : 1 
1400s : 1 
1401 : 1 
1402 : 1 
1402012150 : 1 
1403481 : 1 
1404 : 1 
1405 : 1 
1406 : 1 
1408713398 : 1 
140881861 : 1 
1409 : 1 
1409105377 : 1 
140cdt : 1 
140px : 1 
140x198 : 1 
141 : 1 
141014976 : 1 
1415 : 1 
14159 : 1 
1415926535 : 1 
1416 : 1 
14165 : 1 
1416592423 : 1 
14173 : 1 
1418935413 : 1 
141_reported_by_user : 1 
141st : 1 
142 : 1 
1420 : 1 
1420s : 1 
1421 : 1 
142152748 : 1 
1422 : 1 
142227008 : 1 
1423 : 1 
1423151997 : 1 
1424 : 1 
142458 : 1 
1426 : 1 
1429 : 1 
1429439 : 1 
142_reported_by_user : 1 
142d : 1 
142nd : 1 
143 : 1 
1430s : 1 
1431 : 1 
1433011 : 1 
1435 : 1 
143559040 : 1 
1436 : 1 
1437 : 1 
1437244 : 1 
1438 : 1 
1439 : 1 
143m : 1 
144 : 1 
1440 : 1 
14400484560 : 1 
1440x900 : 1 
1441 : 1 
144552 : 1 
1446 : 1 
144687 : 1 
14469 : 1 
1447 : 1 
1448 : 1

179px : 1 
179th : 1 
17a4 : 1 
17b : 1 
17d : 1 
17drew : 1 
17e : 1 
17g : 1 
17gb : 1 
17iptyvdg8gb8gat5t3ldq : 1 
17ish : 1 
17june : 1 
17kb : 1 
17m : 1 
17n : 1 
17obamawebsub : 1 
17p : 1 
17pholmes : 1 
17s : 1 
17t22 : 1 
17th : 1 
17th_test_squadron : 1 
17the : 1 
17z : 1 
17º : 1 
17½ : 1 
18 : 1 
180 : 1 
1800 : 1 
18000 : 1 
1800060570 : 1 
180062 : 1 
1800824 : 1 
1800s : 1 
1800z : 1 
1801 : 1 
1801s : 1 
1802 : 1 
1803 : 1 
18037011 : 1 
1804 : 1 
1804751393 : 1 
1805 : 1 
1806 : 1 
18066 : 1 
1807 : 1 
1808 : 1 
180802 : 1 
1809 : 1 
18092 : 1 
180px : 1 
180th : 1 
181 : 1 
1810 : 1 
18103193 : 1 
1810s : 1 
1811 : 1 
181195628 : 1 
1812 : 1 
181213623 : 1 
1813 : 1 
181313amsterdam : 1 
1814 : 1 
18140990 : 1 
18141329 : 1 
1815 : 1 
1815823 : 1 
1816 : 1 
181683729 : 1 
1817 : 1 
1818 : 1 
1819 : 1 
18194787 : 1 
18196 : 1 
181981431 : 1 
182 : 1 
1820 : 1 
18208438 : 1 
1820s : 1 
1821 : 1 
1822 : 1 
1823 : 1 
1824 : 1 
1825 : 1 
1826 : 1 
1827 : 1 
1828 : 1 
182

2000mg : 2 
2000s : 2 
2000s_european_sovereign_debt_crisis_timeline : 2 
2000spie : 2 
2000z : 2 
2001 : 2 
200101453405 : 2 
20010321014745 : 2 
2001696254 : 2 
2001810 : 2 
2001a : 2 
2001b : 2 
2001spie : 2 
2002 : 2 
20020421154909 : 2 
20020421155331 : 2 
20020906 : 2 
20024 : 2 
2002_gujarat_violence : 2 
2002brock : 2 
2002opten : 2 
2002tabdate : 2 
2003 : 2 
200303 : 2 
2003041800080100 : 2 
20030629161605 : 2 
20030721baker : 2 
200308 : 2 
20032003 : 2 
2003335773 : 2 
2003752165_andrewhsu18m : 2 
2003_08_09 : 2 
2003in : 2 
2004 : 2 
20040401 : 2 
20040528_classicalbritawardsrm : 2 
20041031053411 : 2 
20042004 : 2 
2004_081 : 2 
2004_pg7_37 : 2 
2004a : 2 
2004bonnie : 2 
2004charley : 2 
2005 : 2 
20050106215257 : 2 
20050128 : 2 
20050502 : 2 
20050608 : 2 
200507 : 2 
2005090409990300 : 2 
20050906 : 2 
20050915130632886 : 2 
20051026 : 2 
20051200210003144 : 2 
200512090016 : 2 
20052005 : 2 
20052012 : 2 
20059618 : 2 
2005a : 2 
2005b : 2 
2005mar8 : 2 
2006 : 2 
20

...

Word kildonan caused exception, returned []
kile : k 
kiled : killed 
kilen : k 
kilfenora : lenora 
kilgallen : ki 
kilgannon : gannon 
kilgarvan : vulgarian 
kilgour : kilgore 
kiliaen : kilian 
kilimandscharo : Kilimanjaro 
kilinochchiya : inclination 
kilkenny : k 
kill : k 
killa : kill a 
killah : kill ah 
killalot : killalea 
killamanjario : Kilimanjaro 
killara : klara 
killcam : kill cam 
kille : k 
killed : k 
killedar : killer 
killedcalifornia71florida20arizona18texas17georgia14new : k 
killeen : killen 
killer : k 
killerboyratz : collaborative 
killerchihua : hierarchical 
killerchihuaha : chihuahua 
killerchihuahau : chihuahua 
killerchihuahua : killer chihuahua 
killerchihuahuah : chihuahua 
killers : k 
killerstartups : killer startups 
killesbergpark : painkiller 
killian : k 
killin : killing 
killind : kill ind 
killiney : killinger 
killing : k 
killingmylobster : billingsgate 
killings : k 
killiondude : Klondike 
killisport : disport 
killjoy : k 
killled : k 
ki

kittybrewster : Brewster 
kitushi : ki 
kitwench : kit wench 
kity : kitty 
kitzinger : k 
kitzmiller : k 
kitzs : kits 
kitāb : kit 
kiudding : kidding 
kiumars : kumar 
kiusatus : katus 
kiusaus : ki 
kivalina_v : k 
kivalliq : ki 
kivel : kiel 
kivler : killer 
kivul : kill 
kiwi : k 
kiwicupache : workplace 
kiwiexile : kiwi exile 
kiwis : k 
kix : k ix 
kiya : k 
Word kiyarr caused exception, returned []
kiyonaga : siyona 
Word kiyosaki caused exception, returned []
kiyoweap : weaponry 
kizz : izz 
kizzle : fizzle 
kizzy : izzy 
kj : k 
kjaer : kjer 
kjasd : ojas 
kjellelec : kelleher 
kjet : k jet 
kjfdsa52talk : k 
kjhk : khaki 
kjoonlee : jolee 
kjust : k just 
kjv : kanv 
kjvo : kjos 
kjvos : kjos 
kjwksnnpzyy : Jacksonian 
kjærlighetens : enlightens 
kjøbenhavns : objectiveness 
kjøtere : kjer 
kk : kuk 
kk277 : k 
kk45 : k 
kkat : kat 
kkeep : k keep 
kker : Fokker 
kkhamenei : Tutankhamen 
kkimchi : kimich 
kking2013 : k 
kkk : kuk 
kkkarrington : arrington 
kkkdoes : does 

knowsuch : know such 
knowthat : know that 
knowwhat : know what 
knowwhere : know where 
knowwing : know wing 
knowyourmeme : nonmember 
knox : k 
knoxville : Knoxville 
knoxx : knox 
knoxy : knox 
knoydart : knoy 
knoz : kn oz 
knpb : knob 
knss : kiss 
knssindia : kassandra 
knub : k nub 
knuckle : k 
knuckleball : k 
knucklehead : k 
knuckleheads : k 
knuckles : k 
knucklesxrouge : knuckleduster 
knuckling : k 
knulla : kulla 
knut : k nut 
knut_hamsun : k 
knuth : knauth 
knutson : k 
knuz : kunz 
knw : know 
knwa : kowa 
knwo : know 
knwoeldge : knowledge 
knwon : kn won 
knws : knows 
knx : knox 
knyaz : kanya 
knysh : knish 
knäckebröd : encumbered 
ko : koy 
koa : kola 
koach : kovach 
koala : k 
koalas : k 
Word koalorka caused exception, returned []
koans : koons 
koavf : kopf 
koay : kocay 
kob : knob 
kobashi : kashi 
kobayashi : Kobayashi 
kobe : kobet 
kober : k 
koblenz : koble 
kobo : koo 
koboi : koo 
Word kobrakid caused exception, returned []
kobres : kore 
kobus : 

korans : korlan 
koratane : korte 
Word korazym caused exception, returned []
korca : kora 
korczak : kozak 
korda : koryda 
kordell : korell 
kordić : kordick 
kore : k 
korea : kora 
korea4expats : k 
korea_under_japanese_rule : k 
koreabang : Korean 
koreaherald : herald 
Word koreajoongangdaily caused exception, returned []
korean : koren 
korean_peninsula : k 
koreanism : organism 
koreanname : forename 
koreanness : kore 
koreans : Koreans 
korelois : kreis 
korenstein : koren 
koresh : koresch 
korg : kort 
korgen : koren 
Word korikatta caused exception, returned []
korismo : korson 
korka : kora 
korku : kort 
kormakitis : dermatitis 
korn : k 
Word kornblatt caused exception, returned []
kornbluh : kornbauer 
korner : koerner 
kornfan71 : k 
korni : korn 
korniloff : kornhoff 
korniyenko : korenko 
koro : koo 
Word korolev caused exception, returned []
Word koroli caused exception, returned []
korolyov : karoly 
koromon : kroon 
koron : korzon 
korona : corona 
kororima : Por

kristjan : kristan 
kristof : kristos 
kristoffer : Kristoffer 
kristy : k 
krisviminda : discriminate 
krit : kritz 
kritik : krithik 
kritisierte : Britisher 
kritzinger : kitzinger 
kriv : krivi 
krivatsy : privratsky 
krivichs : krivi 
krivit : krivi 
krivo : krivi 
krivocheev : microfiche 
krivorogian : Kevorkian 
krivosheev : Kristopher 
kriya : k 
kriyaban : kriya 
kriyabans : crabbiness 
krizan : kirian 
krizdominique : Dominique 
Word krizette caused exception, returned []
krizhevsky : Izhevsk 
krizvinia : kristina 
krizz : kritz 
krk : kirk 
krl : karl 
krleza : reza 
krleža : kleba 
krm500 : k 
krm550 : k 
krmsq : krams 
kro2 : k 
kroatische : grotesque 
kroatischen : kroeschen 
krobertj : robert 
krobin : k robin 
krochmal : kochman 
krocht : kracht 
kroeber : kroeger 
kroeger : k 
kroes : k roes 
krogh : kroh 
krohcol : kroh 
kroker : k 
krokhamlyuk : Yokohama 
krokhmaluk : homologous 
krokhmalyuk : Yokohama 
krokodeilopolis : oligopolistic 
Word krokodopolis caused except

In [32]:
myword='krzyżowcy'
print(fix_spellings(myword))
print(get_best_candidates(myword))
print(norvig_candidates(myword))

krzyżowcy
[]
['krzyżowcy']


In [38]:
for textblock in train_data[:10]:
    print("\n=============================\n")
    print(textblock)
    print("Fixing to")
    print(fix_spellings(textblock))



Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27
Fixing to
Explanation Why the edits made under my user name Hardcore Metallic a Fan were reverted They weren't vandalisms just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27


D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)
Fixing to
Daw He matches this background colour I'm seemingly stuck with. Thanks. talk 21:51, January 11, 2016 UTC


Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actua

In [ ]:
import datetime
for counter, traindata in train_df.iterrows():
    try:
        traindata["comment_text"] = fix_spellings(traindata["comment_text"])
        if counter % 100 == 1:
            print("%s %d processed" % (str(datetime.datetime.now().time()), counter))
    except:
        print('Counter %d caused exception, returned %s' % (counter, traindata["comment_text"]))

14:46:56.342897 1 processed
14:48:22.802918 101 processed
14:49:40.377986 201 processed
14:50:47.018452 301 processed
14:51:44.764665 401 processed
14:52:48.326649 501 processed
14:54:03.767321 601 processed
14:55:21.581634 701 processed
Counter 733 caused exception, returned "
{| width=""100%"" style=""background:transparent""
{| width=""100%"" style=""background-color:#""
|style=""width: 50%; border:1px solid #; background-color:#; vertical-align:top""|
 Hello Madstar890, and Welcome to Wikipedia!
Please remember to sign your name on talk pages by clicking  or using four tildes (~~~~); this will automatically produce your username and the date. Also, please do your best to always fill in the edit summary field. Below are some useful links to facilitate your involvement.
Happy editing! —αἰτίας •discussion• 
{| width=""100%"" style=""background-color:#""
|style=""width: 50%; border:0; background-color:#; vertical-align:top""|
 Getting started Introduction
 The five pillars of Wikipedia

15:06:25.243524 1801 processed
15:55:04.058043 1901 processed
Counter 1907 caused exception, returned Hi, 牛岩, and welcome to Wikipedia! I hope you like the place and decide to stay. Here are some pages that you might find helpful:

Introduction to Wikipedia
The five pillars of Wikipedia
How to edit a page and How to develop articles
How to create your first article 
Simplified Manual of Style

Please remember to sign your messages on talk pages by typing four tildes (~~~~); this will automatically insert your username and the date. If you need help, check out Wikipedia:Questions, ask me on my talk page, or ask your question on this page and then place {{help me}} before the question.

However, there is a small issue with your username. As it is in a non-Latin alphabet script it may not display correctly for many other users. Although usernames like yours are not prohibited, Wikipedia's  signature guideline and username policy encourage you, as a courtesy to other users, to alter your s

16:07:42.520519 3101 processed
16:08:41.753696 3201 processed
16:09:40.771272 3301 processed
Counter 3352 caused exception, returned "

 The Truth about the 1986 Sinn Féin Ard Fheis 

The story of how Gerry Adams tried to turn an eighty year old revolutionary movement into a British Constitutional party.  How he broke the Sinn Féin constitution, created fake cumainn to give him fake votes and barred life long republicans from voting.  How he managed to expel himself and his supporters from Sinn Féin membership.  And, how a small band of republicans managed to keep the Sinn Féin constitution and traditional policy in tact. 

In 1986 Section 1b. of the Sinn Féin constitution read as follows:

“No person who is a member of any political party organisation or who approves of or supports the candidature of persons who, if elected, intend taking part in the proceedings of the Westminster or partitionist 26-County or 6-County  parliaments or who approves of or supports the candidature of pers

16:23:27.992526 4601 processed
16:24:34.737600 4701 processed
16:25:43.071502 4801 processed
Counter 4819 caused exception, returned "::Who's that? Isn't he our alleged leader who is allegedly interested in producing content?♦  

"
16:26:44.399956 4901 processed
16:27:45.177175 5001 processed
Counter 5071 caused exception, returned The definition for Fundamental Attribution Error that is in quotation marks may be bit too much for people unfamiliar with the terms/psychology that is included in the definition. I would maybe replace words like “attributor” and add examples after both “situational factors” and “dispositional factors”. (i.e., “situational factors such as luck,) 
Compared to the Fundamental Attribution Error definition that was provided in quotes, the one provided for attribution theory seems more simplistic and easy to follow, though “social perceiver” may have the same problem as “attributor” did like I mentioned above. Maybe for both definitions you could only use bits an

16:34:40.906662 5601 processed
16:35:32.159628 5701 processed
Counter 5784 caused exception, returned Repeated copyright violations 

You have once again violated copyright by pasting in the contents of http://www.123marseille.com/?p=1〈=en . If you do this again, your account will be blocked from further editing.—
16:36:28.238254 5801 processed
16:37:31.428487 5901 processed
16:38:20.236216 6001 processed
16:39:17.345523 6101 processed
Counter 6178 caused exception, returned Actually, if I can dig through the Wayback machine links, I think they do. It's a matter of manually searching through 200 pages, though, which I'm not going to get done quickly.—
16:40:24.166078 6201 processed
16:41:18.614195 6301 processed
16:43:00.865918 6401 processed
Counter 6482 caused exception, returned "

Saint Patrick's Day is only a week from Saturday… —  $PЯINGrαgђ  "
16:44:06.967100 6501 processed
16:45:18.624549 6601 processed
16:46:26.318718 6701 processed
Counter 6711 caused exception, returned a co

16:49:13.462071 7001 processed
16:50:12.670557 7101 processed
Counter 7124 caused exception, returned History 396 outline for Grey Owl

Career:
1. Trapper (sub heading under career)
His earlier mindset surrounding trapping will be discussed in greater depth, as not much is discussed with regards to how he viewed trapping before his switch to conservationism.
His work with the Ojibwa Indians / Guppy Family will be explored as they were critical in his development as a trapper that understood the fragility of the animal ecosystem (Smith, 41)
The time of his life spent working and living at the Temagami Inn as a chore-boy will be mentioned as this was one of the events in which  Archie first became an observer of the Objibwa way of life. (Smith, 40)
His love interest, Angele will also be discussed as she also introduced Archie to influential figures within the Objibwa community, providing Archie with plenty of sketches for his infamous notebooks. (Smith, 42)

Braz, Albert. “St. Archie of 

Counter 7249 caused exception, returned "== Cohanim J2 - Eleazar - Phinchas - Zadok ==

 Agree. Dr. Karl Skorecki have the final word about Cohanim genetic signatures. He, more than anyone else, is the most credible person that can publish articles and true informations about Cohanim genetic signatures, not Jheald. He was the one who discovered the CMH in 1997. After 10 years, he came out to announce in 2007, that ""he and his research team have discovered not one but two Cohen Modal Haplotypes, which he called J1 and J2"". Katz, Kaplan, Rappaport, Shapiro, Kovacs,Konh, are all J2. Those surnames and families are the most prestigious Cohanim families today. They are all Cohanim J2. J2 Cohanim is much more common than J1 Cohanim. ""Skorecki reported that Pinchas the zealot mentioned in the Bible may be the origin of J2"" This is a very important information that confirms matematical, historic, and genealogy facts. Phinchas was the son of Eleazar. Itamar was his brother. Out of the 24 Co

16:54:58.140146 7501 processed
Counter 7582 caused exception, returned "

Elliott Carter inappropriate?
I see that  has tagged Elliott Carter's name as ""inappropriate"" under the ""Opera"" heading in this article. What is inappropriate about Mr. Carter in this context?—  "
16:55:40.767270 7601 processed
Counter 7610 caused exception, returned Yep, that all makes perfect sense, and the move is uncontroversial with me and the page's creator, with whom I discussed this a bit on the Talk page when I made my initial, ill-fated moves.  Just to clarify what I was after with singular v. plural (so as not to appear a plodding dolt), I meant that if we were to push an article toward being about a genre in which works can be referred to in the singular or plural, the singular seems to be the preferred usage in discussing genre.  A point of general interest: the other, singular pinakes listed in the Suda entry aren't considered parts of the work known as the Pinakes, that being Πίνακες τῶν ἐν πάσ

17:00:06.805412 8001 processed
17:01:12.069648 8101 processed
17:02:18.471399 8201 processed
17:04:32.022431 8301 processed
17:05:42.469683 8401 processed
17:06:45.747726 8501 processed
17:07:47.451849 8601 processed
Counter 8645 caused exception, returned Do the following two statements refer to the same event?

On April 5, Serbian policemen attacked police stations and then an Interior Ministry training school.
Also on April 6 Yugoslav People's Army (JNA) forces, Serb police units and paramilitaries attacked the Bosnian government’s personnel training center.

If so, we should note it just once, and get the participants right; and also we need to establish the correct date.
17:08:32.406842 8701 processed
17:09:33.349197 8801 processed
17:10:40.857736 8901 processed
Counter 8949 caused exception, returned Do research...
useful links: http://www.google.co.uk/search?q=%22Centre+for+Appropriate+Technology%22+canada&start;=0&ie;=utf-8&oe;=utf-8

One Sky - The Canadian Institute of Sustain

17:38:06.948949 11301 processed
17:38:58.205751 11401 processed
17:40:34.567650 11501 processed
Counter 11556 caused exception, returned Controversies - Did he had conspired 9/11? 

I don't want to make Wikipedia offended (primarily the article), but I think it's strange that he had financed to implode the buildings on the World Trade Center. I don't think it would be relevant in the article by Wikipedia's standards or if I would receive a notification on disruption or something like that. Here I go. 

He is a Jewish person right? He wanted to implode the buildings for a war on Islam. Secondly, there was a television tower on top of the World Trade Center - does that seem weird to you people? That was all done for capitalism, propaganda, scare tactics, exploitation and viewers? No one talks about that. The television agencies wanted them to blame Islam for good measure, because of oil.  Oil, television agencies, government power, and lobbyist, in a government operated building. This ma